# Lymphoid_No4-LV-step1_edgeR_DEgenes

## Import required modules

In [1]:
import os
import anndata
import logging
import numpy as np
import pandas as pd
from datetime import timedelta, datetime
import scanpy as sc
import seaborn as sns
import matplotlib.pyplot as plt
import leidenalg
import csv
from pandas import Timestamp
import re
import matplotlib.pyplot as plt
from scipy.stats import rankdata 
import scipy
import harmonypy as hm
import random
from scipy.stats.stats import pearsonr   
import scipy.sparse
from SCCAF import *

In [2]:
sc.logging.print_header()
sc.settings.verbosity = 3
sc.settings.set_figure_params(dpi=140, color_map='RdBu_r')

scanpy==1.6.0 anndata==0.7.4 umap==0.3.10 numpy==1.18.2 scipy==1.4.1 pandas==1.0.4 scikit-learn==0.22.2.post1 statsmodels==0.12.2 python-igraph==0.9.0 louvain==0.7.0 leidenalg==0.8.0


In [3]:
%load_ext rpy2.ipython

## Paths

In [4]:
path_out = '/home/jovyan/projects/kk14_DCM-lymphoid/results/for_edgeR/210526/'

## Functions

In [5]:
def correlation_analysis(scanpy_object, column_tosplitby, sum_or_mean):
    d = {}
    for cluster_number in np.unique(scanpy_object.obs[column_tosplitby].values):
        #cluster_number=np.unique(heart.obs['louvain_annotated'].values)[1]
        scanpy_object_subset = scanpy_object[scanpy_object.obs[column_tosplitby].isin([cluster_number])]

        if sum_or_mean=="mean":
            d[cluster_number] = np.squeeze(np.asarray(scanpy_object_subset.raw.X.mean(axis=0))) #np.log(+1)
        elif sum_or_mean=="sum":
            d[cluster_number] = np.squeeze(np.asarray(scanpy_object_subset.raw.X.sum(axis=0))) #np.log(+1)
        del scanpy_object_subset
        
    return d

## Read dataset

### read global raw

In [6]:
global_all = sc.read_h5ad('/nfs/team205/kk18/DCM/anndata/global_all_ANNOTATED_CELLSTATES_RAW_V3.h5ad')
global_all

/home/jovyan/my-conda-envs/kk14-env/lib/python3.8/site-packages/anndata/_core/anndata.py:119: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


AnnData object with n_obs × n_vars = 881081 × 33538
    obs: 'Barcode', 'Patient', 'Sample', 'Gender', 'Gene', 'Diagnosis', 'Clinical.dominant.mutation', 'Age', 'Mutation.Type', 'Genomic.location', 'Region', 'Origin', 'n_genes', 'percent_mito', 'percent_ribo', 'X10X_version', 'n_counts', 'solo_score', 'predicted_doublets_solo', 'scrublet_score_z', 'predicted_doublets_z', 'scrublet_score_log', 'predicted_doublets_log', 'Replicate', 'cell_states_20200602', 'cell_type_20200602', 'cell_type_20200921', 'cell_type_global_HCA', 'cell_states_HCA', 'cell_type_fine_20200921', 'cell_states_20200921', 'cell_states_long_20200921', 'cell_states_20210218', 'validated_20210218', 'leiden_01', 'leiden_02', 'leiden_03', 'leiden_04', 'cell_type', 'cell_states_20210302', 'cell_states_long', 'validated', 'cell_type_toplot1', 'cell_type_toplot2', 'cell_type_toplot3', 'cell_type_toplot4', 'cell_states'
    var: 'gene_ids', 'feature_types', 'genome'

In [7]:
subs = {
    "vEC":"EC",
    "nonvEC":"EC", 
    "SMC":"Mural", 
    "PC":"Mural", 
     } 
# PC-like, EC-like, MP-like, MP_interacting, CM-like, ECM_r, CST3+, ECM_p, Canonical, DAAM1+
x=[subs.get(item,item)  for item in global_all.obs['cell_type']]
global_all.obs['cell_type']=x
pd.value_counts(global_all.obs['cell_type'])/global_all.shape[0]

CM          0.358217
Mural       0.199124
FB          0.176731
EC          0.145108
Myeloid     0.079489
Lymphoid    0.022796
N           0.011461
Mast        0.003926
AD          0.003150
Name: cell_type, dtype: float64

In [8]:
set(global_all.obs[global_all.obs['cell_type']=='Lymphoid']['cell_states'])

{'B',
 'CD4T_act',
 'CD4T_naive',
 'CD4T_reg',
 'CD8T_cytox',
 'CD8T_em',
 'CD8T_te',
 'CD8T_te_IFNGhi',
 'CD8T_trans',
 'ILC',
 'LY0',
 'MAIT-like',
 'NK_CD16hi',
 'NK_CD16hiIFNGhi',
 'NK_CD56hi',
 'Plasma',
 'prolif_Lymphoids',
 'unclassified.1',
 'unclassified.2'}

In [9]:
global_all=global_all[global_all.obs.Region.isin(['FW', 'AP', 'S'])]
global_all=global_all[global_all.obs.validated]
global_all=global_all[global_all.obs['Clinical.dominant.mutation'].isin(['TTN', 'LMNA', 'RBM20', 'PKP2', 'control', 'PVneg'])]
global_all

View of AnnData object with n_obs × n_vars = 525840 × 33538
    obs: 'Barcode', 'Patient', 'Sample', 'Gender', 'Gene', 'Diagnosis', 'Clinical.dominant.mutation', 'Age', 'Mutation.Type', 'Genomic.location', 'Region', 'Origin', 'n_genes', 'percent_mito', 'percent_ribo', 'X10X_version', 'n_counts', 'solo_score', 'predicted_doublets_solo', 'scrublet_score_z', 'predicted_doublets_z', 'scrublet_score_log', 'predicted_doublets_log', 'Replicate', 'cell_states_20200602', 'cell_type_20200602', 'cell_type_20200921', 'cell_type_global_HCA', 'cell_states_HCA', 'cell_type_fine_20200921', 'cell_states_20200921', 'cell_states_long_20200921', 'cell_states_20210218', 'validated_20210218', 'leiden_01', 'leiden_02', 'leiden_03', 'leiden_04', 'cell_type', 'cell_states_20210302', 'cell_states_long', 'validated', 'cell_type_toplot1', 'cell_type_toplot2', 'cell_type_toplot3', 'cell_type_toplot4', 'cell_states'
    var: 'gene_ids', 'feature_types', 'genome'

In [10]:
def paste0(i, j, k, l, m):
    return(str(i)+"__"+str(j)+"__"+str(k)+"__"+str(l)+"__"+str(m))


global_all.obs['pool_key']=[paste0(i,j,k,l,m) for i,j,k,l,m in zip(global_all.obs['Clinical.dominant.mutation'],
                                                                   global_all.obs['cell_states'],
                                                                   global_all.obs['Patient'],
                                                                   global_all.obs['X10X_version'],
                                                                   global_all.obs['Gender'])]

Trying to set attribute `.obs` of view, copying.


In [11]:
global_all.raw=global_all
x=correlation_analysis(global_all, 'pool_key', 'sum')
x=pd.DataFrame(x)
x.index=global_all.var_names
x.head()

,LMNA__AD1.0__DL2__V3__m,LMNA__AD1.0__H05__V3__m,LMNA__AD1.0__H26__V3__f,LMNA__AD1.0__H28__V3__m,LMNA__AD1.0__H31__V3__m,LMNA__AD1.1__H05__V3__m,LMNA__AD1.1__H06__V3__f,LMNA__AD1.1__H24__V3__f,LMNA__AD1.1__H25__V3__m,LMNA__AD1.1__H26__V3__f,...,control__vFB4__D4__V2__f,control__vFB4__D5__V2__f,control__vFB4__D6__V2__m,control__vFB4__D7__V2__m,control__vFB4__H46__V3__m,control__vFB4__H49__V3__f,control__vFB4__H51__V3__m,control__vFB4__H53__V3__m,control__vFB4__H55__V3__m,control__vFB4__H67__V3__m
MIR1302-2HG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
FAM138A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
OR4F5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AL627309.1,0.0,9.0,0.0,2.0,0.0,3.0,0.0,0.0,1.0,0.0,...,2.0,2.0,0.0,0.0,2.0,0.0,4.0,2.0,3.0,4.0
AL627309.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
x.to_csv(path_out + "ALL_CELLSTATE_PSEUDOBULK_LVS.csv")

## for filter genes

In [13]:
global_all.obs['Gene_Patient']=[str(i) + "_" + str(j) + "_" + str(k) for i,j,k in zip(global_all.obs['Clinical.dominant.mutation'], 
                                                                                      global_all.obs['cell_states'],
                                                                                      global_all.obs['Patient'])]

In [14]:
subset_=sc.AnnData(global_all[:,global_all.var.index].raw.X, # 
                  obs=global_all.obs,
                  var=global_all[:,global_all.var.index].raw.var
                 )
subset_

AnnData object with n_obs × n_vars = 525840 × 33538
    obs: 'Barcode', 'Patient', 'Sample', 'Gender', 'Gene', 'Diagnosis', 'Clinical.dominant.mutation', 'Age', 'Mutation.Type', 'Genomic.location', 'Region', 'Origin', 'n_genes', 'percent_mito', 'percent_ribo', 'X10X_version', 'n_counts', 'solo_score', 'predicted_doublets_solo', 'scrublet_score_z', 'predicted_doublets_z', 'scrublet_score_log', 'predicted_doublets_log', 'Replicate', 'cell_states_20200602', 'cell_type_20200602', 'cell_type_20200921', 'cell_type_global_HCA', 'cell_states_HCA', 'cell_type_fine_20200921', 'cell_states_20200921', 'cell_states_long_20200921', 'cell_states_20210218', 'validated_20210218', 'leiden_01', 'leiden_02', 'leiden_03', 'leiden_04', 'cell_type', 'cell_states_20210302', 'cell_states_long', 'validated', 'cell_type_toplot1', 'cell_type_toplot2', 'cell_type_toplot3', 'cell_type_toplot4', 'cell_states', 'pool_key', 'Gene_Patient'
    var: 'gene_ids', 'feature_types', 'genome'

In [15]:
subset_.X.data[:10]

array([1., 1., 1., 1., 3., 1., 1., 1., 1., 2.], dtype=float32)

In [16]:
subset_.raw=subset_
x=correlation_analysis(subset_, 'Gene_Patient', 'mean') # MEan per Gene_Celltype_Patient
x=pd.DataFrame(x)
x.index=global_all.var.index

In [17]:
# del global_all

In [18]:
MUTATIONS=[str(x) + "_" + str(y) for x in global_all.obs['Clinical.dominant.mutation'].unique() for 
           y in global_all.obs['cell_states'].unique()]
MUTATIONS

['control_SMC1.2',
 'control_vCM2',
 'control_vCM1.0',
 'control_EC8.0',
 'control_PC1',
 'control_vCM3.0',
 'control_PC3',
 'control_NC2',
 'control_vFB1.0',
 'control_vCM1.1',
 'control_vCM1.2',
 'control_CD4T_act',
 'control_EC1.0',
 'control_vFB4',
 'control_Mast',
 'control_vCM4',
 'control_EC5.0',
 'control_vFB1.1',
 'control_vCM5',
 'control_SMC1.1',
 'control_EC2.0',
 'control_MY12',
 'control_EC6.0',
 'control_Plasma',
 'control_SMC2',
 'control_vFB2',
 'control_PC2',
 'control_vCM1.3',
 'control_CD8T_em',
 'control_vFB1.2',
 'control_MY1',
 'control_NC1.0',
 'control_NC1.1',
 'control_NC5',
 'control_AD2',
 'control_vCM3.1',
 'control_MY2',
 'control_MY3',
 'control_CD8T_trans',
 'control_NC3',
 'control_MY15',
 'control_MY5',
 'control_CD8T_te',
 'control_MAIT-like',
 'control_CD4T_naive',
 'control_MY4',
 'control_AD1.0',
 'control_prolif Myeloids',
 'control_MY14',
 'control_cDC1',
 'control_MY7',
 'control_NC4',
 'control_cDC2',
 'control_NC1.4',
 'control_NC1.2',
 'contr

In [19]:
pre=[]
for MUTATION in MUTATIONS:
    OUT=np.where([re.match(MUTATION, COLNAME) for COLNAME in x.columns])[0]
    pre.append(OUT)
pre

[array([3990, 3991, 3992, 3993, 3994, 3995, 3996, 3997, 3998, 3999, 4000,
        4001, 4002, 4003, 4004, 4005, 4006, 4007]),
 array([4171, 4172, 4173, 4174, 4175, 4176, 4177, 4178, 4179, 4180, 4181,
        4182, 4183, 4184, 4185, 4186, 4187, 4188]),
 array([4099, 4100, 4101, 4102, 4103, 4104, 4105, 4106, 4107, 4108, 4109,
        4110, 4111, 4112, 4113, 4114, 4115, 4116]),
 array([3404, 3405, 3406, 3407, 3408, 3409, 3410, 3411, 3412, 3413, 3414,
        3415, 3416, 3417, 3418, 3419, 3420, 3421]),
 array([3906, 3907, 3908, 3909, 3910, 3911, 3912, 3913, 3914, 3915, 3916,
        3917, 3918, 3919, 3920, 3921, 3922, 3923]),
 array([4189, 4190, 4191, 4192, 4193, 4194, 4195, 4196, 4197, 4198, 4199,
        4200, 4201, 4202, 4203, 4204, 4205, 4206]),
 array([3942, 3943, 3944, 3945, 3946, 3947, 3948, 3949, 3950, 3951, 3952,
        3953, 3954, 3955, 3956, 3957, 3958, 3959]),
 array([3787, 3788, 3789, 3790, 3791, 3792, 3793, 3794, 3795, 3796, 3797,
        3798, 3799, 3800, 3801, 3802]),
 arr

In [20]:
from fastprogress.fastprogress import  progress_bar

In [21]:
LIST_COLNAMES=[]
for MUTATION in progress_bar(MUTATIONS):
    otherMUTATIONS = list(set(MUTATIONS) - set([MUTATION]))
    OUT=np.array([i for i in range(len(x.columns)) \
                  if (MUTATION in x.columns[i]) & \
                     (all([mut not in x.columns[i] for mut in otherMUTATIONS if mut not in MUTATION])) # for specificity (e.g. for 'B' and 'B_plasma')
                 ]
                )
    LIST_COLNAMES.append(OUT)

x=[x.iloc[:,i].mean(axis=1) for i in LIST_COLNAMES] # Mean per Genotype_Cellstate
x=pd.concat(x, axis=1)

In [22]:
len(LIST_COLNAMES)

474

In [23]:
for i in range(len(LIST_COLNAMES)):
    if len(pre[i]) == len(LIST_COLNAMES[i]):
        if all(LIST_COLNAMES[i]==pre[i]):
            pass
        else:
            print(MUTATIONS[i])
    else:
        print(MUTATIONS[i])

control_MY1
control_CD8T_te
control_NK_CD16hi
control_CD16+ Mo
TTN_MY1
TTN_CD8T_te
TTN_NK_CD16hi
TTN_CD16+ Mo
LMNA_MY1
LMNA_CD8T_te
LMNA_NK_CD16hi
LMNA_CD16+ Mo
RBM20_MY1
RBM20_CD8T_te
RBM20_NK_CD16hi
RBM20_CD16+ Mo
PKP2_MY1
PKP2_CD8T_te
PKP2_NK_CD16hi
PKP2_CD16+ Mo
PVneg_MY1
PVneg_CD8T_te
PVneg_NK_CD16hi
PVneg_CD16+ Mo


In [24]:
pre[i]

array([], dtype=int64)

In [25]:
LIST_COLNAMES[i]

array([], dtype=float64)

In [26]:
len(pre[0])

18

In [27]:
x.columns=MUTATIONS
x.head()

,control_SMC1.2,control_vCM2,control_vCM1.0,control_EC8.0,control_PC1,control_vCM3.0,control_PC3,control_NC2,control_vFB1.0,control_vCM1.1,...,PVneg_Meso,PVneg_NC6,PVneg_NC1.3,PVneg_CD4T_reg,PVneg_NK_CD16hiIFNGhi,PVneg_CD8T_te_IFNGhi,PVneg_unclassified.2,PVneg_AD1.1,PVneg_unclassified.1,PVneg_AD3
MIR1302-2HG,0.000000,0.000077,0.000024,0.000000,0.000000,0.000209,0.000000,0.000000,0.000133,0.000264,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,NaN
FAM138A,0.000000,0.000047,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,NaN
OR4F5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,NaN
AL627309.1,0.006045,0.037665,0.021153,0.018519,0.005672,0.026386,0.006669,0.051339,0.011350,0.015490,...,0.0,0.0,0.047619,0.083333,0.0,0.006667,0.0,0.111111,0.0,NaN
AL627309.3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,NaN


In [28]:
x.columns=MUTATIONS
x

,control_SMC1.2,control_vCM2,control_vCM1.0,control_EC8.0,control_PC1,control_vCM3.0,control_PC3,control_NC2,control_vFB1.0,control_vCM1.1,...,PVneg_Meso,PVneg_NC6,PVneg_NC1.3,PVneg_CD4T_reg,PVneg_NK_CD16hiIFNGhi,PVneg_CD8T_te_IFNGhi,PVneg_unclassified.2,PVneg_AD1.1,PVneg_unclassified.1,PVneg_AD3
MIR1302-2HG,0.000000,0.000077,0.000024,0.000000,0.000000,0.000209,0.000000,0.000000,0.000133,0.000264,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,NaN
FAM138A,0.000000,0.000047,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,NaN
OR4F5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,NaN
AL627309.1,0.006045,0.037665,0.021153,0.018519,0.005672,0.026386,0.006669,0.051339,0.011350,0.015490,...,0.0,0.0,0.047619,0.083333,0.0,0.006667,0.0,0.111111,0.0,NaN
AL627309.3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AC233755.2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,NaN
AC233755.1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,NaN
AC240274.1,0.004312,0.007920,0.006346,0.001736,0.004602,0.007490,0.003567,0.004464,0.012138,0.004304,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,NaN
AC213203.1,0.000000,0.000000,0.000000,0.000000,0.000036,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,NaN


In [29]:
x.to_csv(path_out + "ALL_CELLSTATE_PSEUDOBULK_FILTERING_LVS.csv")

In [30]:
x = pd.Series(global_all.obs.cell_type.values,
          index=global_all.obs.cell_states).to_dict()
x=pd.DataFrame.from_dict(x, orient='index')
x[x[0]=='Lymphoid']

,0
CD4T_act,Lymphoid
Plasma,Lymphoid
CD8T_em,Lymphoid
CD8T_trans,Lymphoid
CD8T_te,Lymphoid
MAIT-like,Lymphoid
CD4T_naive,Lymphoid
NK_CD56hi,Lymphoid
NK_CD16hi,Lymphoid
ILC,Lymphoid


In [31]:
x.to_csv(path_out + "CELLSTATE_TRANSLATION_TABLE.csv")

## Pseudobulk for cell_type 

In [32]:
global_all.obs.drop(['pool_key'], axis=1, inplace=True)
global_all.obs.head()

,Barcode,Patient,Sample,Gender,Gene,Diagnosis,Clinical.dominant.mutation,Age,Mutation.Type,Genomic.location,...,cell_type,cell_states_20210302,cell_states_long,validated,cell_type_toplot1,cell_type_toplot2,cell_type_toplot3,cell_type_toplot4,cell_states,Gene_Patient
2428,AAACCCAAGGTAAACT-BS_H25_S0_premrna,BS_H25,BS_H25_S00_premrna,m,control,Control,control,52.0,nan,nan,...,Mural,SMC1.2,SMC1.2_basic,True,PC,PC,PC,PC,SMC1.2,control_SMC1.2_BS_H25
2429,AAACCCAGTTGGTGTT-BS_H25_S0_premrna,BS_H25,BS_H25_S00_premrna,m,control,Control,control,52.0,nan,nan,...,CM,vCM2,vCM2,True,CM,CM,CM,CM,vCM2,control_vCM2_BS_H25
2430,AAACCCATCGACGCGT-BS_H25_S0_premrna,BS_H25,BS_H25_S00_premrna,m,control,Control,control,52.0,nan,nan,...,CM,vCM1.0,vCM1.0,True,CM,CM,CM,CM,vCM1.0,control_vCM1.0_BS_H25
2431,AAACGAAAGGCGCTTC-BS_H25_S0_premrna,BS_H25,BS_H25_S00_premrna,m,control,Control,control,52.0,nan,nan,...,EC,EC8.0,EC8.0_ln,True,nonvEC,nonvEC,nonvEC,nonvEC,EC8.0,control_EC8.0_BS_H25
2432,AAACGAAAGGGCCAAT-BS_H25_S0_premrna,BS_H25,BS_H25_S00_premrna,m,control,Control,control,52.0,nan,nan,...,Mural,PC1,PC1_vent,True,PC,PC,PC,PC,PC1,control_PC1_BS_H25


In [33]:
def paste0(i, j, k, l, m):
    return(str(i)+"__"+str(j)+"__"+str(k)+"__"+str(l)+"__"+str(m))


global_all.obs['pool_key']=[paste0(i,j,k,l,m) for i,j,k,l,m in zip(global_all.obs['Clinical.dominant.mutation'],
                                                                   global_all.obs['cell_type'],
                                                                   global_all.obs['Patient'],
                                                                   global_all.obs['X10X_version'],
                                                                   global_all.obs['Gender'])]

In [34]:
x=correlation_analysis(global_all, 'pool_key', 'sum')
x=pd.DataFrame(x)
x.index=global_all.var.index
x.head()

,LMNA__AD__DL2__V3__m,LMNA__AD__H05__V3__m,LMNA__AD__H06__V3__f,LMNA__AD__H24__V3__f,LMNA__AD__H25__V3__m,LMNA__AD__H26__V3__f,LMNA__AD__H27__V3__f,LMNA__AD__H28__V3__m,LMNA__AD__H29__V3__f,LMNA__AD__H31__V3__m,...,control__N__D4__V2__f,control__N__D5__V2__f,control__N__D6__V2__m,control__N__D7__V2__m,control__N__H46__V3__m,control__N__H49__V3__f,control__N__H51__V3__m,control__N__H53__V3__m,control__N__H55__V3__m,control__N__H67__V3__m
MIR1302-2HG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
FAM138A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
OR4F5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AL627309.1,0.0,19.0,0.0,0.0,1.0,0.0,1.0,14.0,0.0,0.0,...,0.0,1.0,0.0,0.0,2.0,0.0,3.0,1.0,3.0,2.0
AL627309.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
x.to_csv(path_out + "ALL_CELLTYPE_PSEUDOBULK_LVS.csv")

## Abundance for cell_type

In [36]:
x=pd.crosstab(global_all.obs['cell_type'],global_all.obs['Patient'])
y=pd.crosstab(global_all.obs['cell_states'],global_all.obs['Patient'])

In [37]:
x.to_csv(path_out + "ABSOLUTE_CELLTYPE_NUMBER_LVS.csv")
y.to_csv(path_out + "ABSOLUTE_CELLSTATES_NUMBER_LVS.csv")

In [38]:
y

Patient,BS_H15,BS_H20,BS_H25,BS_H26,BS_H35,BS_H37,D1,D2,D4,D5,...,H53,H55,H56,H57,H58,H59,H67,IC_H01,IC_H02,IC_H04
cell_states,,,,,,,,,,,,,,,,,,,,,
AD1.0,21,29,6,52,22,0,54,71,109,66,...,1,0,0,0,0,12,0,0,1,2
AD1.1,8,0,0,18,30,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
AD2,12,2,2,23,33,3,1,1,3,0,...,0,0,0,1,0,1,0,0,0,0
AD3,0,0,0,23,4,0,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
B,2,14,2,0,1,2,0,1,1,1,...,1,2,0,0,3,0,4,0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
vFB1.1,111,59,128,47,59,43,125,377,594,589,...,182,321,84,5,17,34,139,23,38,74
vFB1.2,81,73,35,28,20,34,22,44,111,86,...,48,16,31,15,18,14,45,24,136,0
vFB2,159,363,162,150,107,80,191,298,526,397,...,180,118,148,105,133,157,495,171,251,62
